In [54]:
import pandas as pd
import json
import os
import deepsig
from IPython.display import display

In [55]:
cols = ['dataset', 'method', 'fitness_rule', 'fitness', 'ACC', 'MCC', 'f1_score', 'avg_odds_diff', 'stat_par_diff', 'eq_opp_diff']

In [56]:
mlp_baseline_results = pd.read_csv('mlp_baseline_results.csv')
mlp_baseline_results.replace({'simple_mlp_initializer': 'Standard MLP (baseline)'}, inplace=True)

mlp_pearson_results = pd.read_csv('mlp_pearson_results.csv')
mlp_pearson_results.replace({'mlp_preg_initializer': 'Pearson regularized MLP'}, inplace=True)

mlp_spearman_results = pd.read_csv('mlp_spearman_results.csv')
mlp_spearman_results.replace({'mlp_sreg_initializer': 'Spearman regularized MLP'}, inplace=True)

mlp_auto_reg_results = pd.read_csv('mlp_auto_reg_results.csv')
mlp_auto_reg_results.replace({'mlp_auto_reg_initializer': 'Auto corr regularized MLP'}, inplace=True)

mlp_results = pd.concat([mlp_baseline_results, mlp_pearson_results, mlp_spearman_results, mlp_auto_reg_results])

ftl_baseline_results = pd.read_csv('ftl_baseline_results.csv')
ftl_baseline_results.replace({'ftl_mlp_initializer': 'Standard FTL (baseline)'}, inplace=True)

ftl_pearson_results = pd.read_csv('ftl_pearson_results.csv')
ftl_pearson_results.replace({'ftl_mlp_preg_initializer': 'Pearson regularized FTL'}, inplace=True)

ftl_spearman_results = pd.read_csv('ftl_spearman_results.csv')
ftl_spearman_results.replace({'ftl_mlp_sreg_initializer': 'Spearman regularized FTL'}, inplace=True)

ftl_auto_reg_results = pd.read_csv('ftl_auto_reg_results.csv')
ftl_auto_reg_results.replace({'ftl_mlp_auto_reg_initializer': 'Auto corr regularized FTL'}, inplace=True)

ftl_results = pd.concat([ftl_baseline_results, ftl_pearson_results, ftl_spearman_results, ftl_auto_reg_results])

In [57]:
for results in [mlp_results,ftl_results]:
    results.replace({'adult_dataset_reader': 'Adult Income', 'compas_dataset_reader': 'Compas Recidivism', 'german_dataset_reader': 'German Credit', 'bank_dataset_reader': 'Bank Marketing'}, inplace=True)
    results.rename(columns={'avg_odds_diff': 'Equalized Odds', 'stat_par_diff': 'Statistical Parity', 'eq_opp_diff': 'Equal Opportunity', 'MCC': 'Mathew Correlation', 'ACC': 'Accuracy'}, inplace=True)

In [58]:
fitness_rules_target_metrics = {
    'mcc_parity': {'performance': 'Mathew Correlation', 'fairness': 'Statistical Parity'},
    'mcc_opportunity': {'performance': 'Mathew Correlation', 'fairness': 'Equal Opportunity'},
    'mcc_odds': {'performance': 'Mathew Correlation', 'fairness': 'Equalized Odds'},
    'acc_parity': {'performance': 'Accuracy', 'fairness': 'Statistical Parity'},
    'acc_opportunity': {'performance': 'Accuracy', 'fairness': 'Equal Opportunity'},
    'acc_odds': {'performance': 'Accuracy', 'fairness': 'Equalized Odds'}
}

fitness_rules_target_metrics = {
    'mcc_parity': ('Mathew Correlation', 'Statistical Parity'),
    'mcc_opportunity': ('Mathew Correlation', 'Equal Opportunity'),
    'mcc_odds': ('Mathew Correlation', 'Equalized Odds'),
    'acc_parity': ('Accuracy', 'Statistical Parity'),
    'acc_opportunity': ('Accuracy', 'Equal Opportunity'),
    'acc_odds': ('Accuracy', 'Equalized Odds')
}
fitness_rules_abvr = {
    'mcc_parity': 'Max(MCC - Stat. Parity)',
    'mcc_opportunity': 'Max(MCC - Eq. Odds)',
    'mcc_odds': 'Max(MCC - Eq. Opp.)',
    'acc_parity': 'Max(Acc - Stat. Parity)',
    'acc_opportunity': 'Max(Acc - Eq. Odds)',
    'acc_odds': 'Max(Acc - Eq. Opp.)'
}

for results in [mlp_results,ftl_results]:
    results['Performance'] = 0
    results['Fairness'] = 0
    results['Fitness Rule'] = ''
    for fitness_rule, (performance_metric, fairness_metric) in fitness_rules_target_metrics.items():
        results.loc[results.fitness_rule == fitness_rule,'Performance'] = results.loc[results.fitness_rule == fitness_rule,performance_metric]
        results.loc[results.fitness_rule == fitness_rule,'Fairness'] = results.loc[results.fitness_rule == fitness_rule,fairness_metric]
        results.loc[results.fitness_rule == fitness_rule,'Fitness Rule Abvr'] = fitness_rules_abvr[fitness_rule]
        results.loc[results.fitness_rule == fitness_rule,'Fitness Rule'] = 'Max(%s - %s)' % fitness_rules_target_metrics[fitness_rule]

/tmp/ipykernel_2889562/3607436483.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.58517004 0.58512823 0.5840247  0.5836371  0.57625737 0.5769683
 0.58075151 0.57811729 0.57147862 0.55847176 0.56402881 0.58191431
 0.58343253 0.57740222 0.51579307 0.30958237 0.37962947 0.55899189
 0.53558872 0.2598879  0.57717035 0.50086739 0.33275221 0.37929293
 0.47729109 0.29196869 0.29037704 0.52303609 0.27480633 0.38668781
 0.50626951 0.29960979 0.4042848  0.52119218 0.2936422  0.33522388
 0.49730721 0.25112662 0.35119067 0.510052   0.26843855 0.36081008
 0.52157495 0.30114722 0.42995147 0.48267704 0.29056691 0.23139881
 0.54223416 0.23653437 0.27041017 0.52121263 0.25759348 0.23912165
 0.5223892  0.27406652 0.22303564 0.54094974 0.27699558 0.30939251
 0.51731345 0.30339828 0.37825089 0.52113734 0.26323857 0.33963196
 0.5538225  0.27850228 0.30234984 0.51877126 0.29494101 0.2648939
 0.52742877 0.24492927 0.13407928 0.535

In [59]:
datasets = ['Adult Income', 'Bank Marketing', 'Compas Recidivism','German Credit']
datasets

['Adult Income', 'Bank Marketing', 'Compas Recidivism', 'German Credit']

In [60]:
fitness_rules = ['mcc_parity', 'mcc_opportunity', 'mcc_odds', 'acc_parity', 'acc_opportunity', 'acc_odds']
fitness_rules

['mcc_parity',
 'mcc_opportunity',
 'mcc_odds',
 'acc_parity',
 'acc_opportunity',
 'acc_odds']

In [61]:
ftl_methods = ['Standard FTL (baseline)', 'Pearson regularized FTL', 'Spearman regularized FTL', 'Auto corr regularized FTL']
mlp_methods = ['Standard MLP (baseline)', 'Pearson regularized MLP', 'Spearman regularized MLP', 'Auto corr regularized MLP']
significances = []
grouped_results_list = []

In [62]:
for path, methods, results in zip(['mlp_multi_aso_data_list.json', 'ftl_multi_aso_data_list.json'],
                                  [mlp_methods, ftl_methods],
                                  (mlp_results,ftl_results)):
    if os.path.exists(path):
        with open(path) as file:
            multi_aso_data_list = json.load(file)
    else:    
        multi_aso_data_list = []
        for d in datasets:
            multi_aso_data = []
            for f in fitness_rules:
                methods_results = []
                for m in methods:
                    r = results.loc[ (results['dataset'] == d) &
                                         (results['fitness_rule'] == f) &
                                         (results['method'] == m) ]\
                                .fitness.tolist()
                    if len(r) == 0:
                        r = [-1]
                    methods_results.append(r)
                min_eps = deepsig.multi_aso(methods_results, confidence_level=0.95)
                multi_aso_data_list.append({'fitness_rule': f, 'dataset': d, 'min_eps': min_eps.tolist()})
        with open(path, 'w') as file:
            json.dump(multi_aso_data_list, file)
    
    aso_df_resume = []
    for aso_result in sorted(multi_aso_data_list, key=lambda x: x['dataset']):
        fitness_rule = aso_result['fitness_rule']
        dataset = aso_result['dataset']
    
        aso_df = pd.DataFrame(aso_result['min_eps'], columns=methods)
        aso_df['method'] = methods
        aso_df['dataset'] = dataset
        aso_df['fitness_rule'] = fitness_rule
        aso_df_resume.append(aso_df)
    
    print('Significance Testing')
    significance = pd.concat(aso_df_resume)
    significance.replace(fitness_rules_abvr, inplace=True)
    significance = significance.set_index(['fitness_rule', 'dataset'])
    significance = significance.sort_values(by=['fitness_rule', 'dataset'], ascending=[False, True])
    significances.append(significance)
    
    grouped_results = results\
        .groupby(['Fitness Rule Abvr', 'dataset', 'method'])\
        .agg({'fitness': ['mean', 'std', 'count'], 'Performance': ['mean', 'std'], 'Fairness': ['mean', 'std']})\
        .sort_values(by=['Fitness Rule Abvr', 'dataset', ('fitness','mean')], ascending=False)
    grouped_results['formatted_fitness'] = grouped_results.apply(lambda row: f"${row[('fitness', 'mean')]:.3f} (\pm{row[('fitness', 'std')]:.2f})$", axis=1)
    grouped_results['formatted_performance'] = grouped_results.apply(lambda row: f"${row[('Performance', 'mean')]:.3f} (\pm{row[('Performance', 'std')]:.2f})$", axis=1)
    grouped_results['formatted_fairness'] = grouped_results.apply(lambda row: f"${row[('Fairness', 'mean')]:.3f} (\pm{row[('Fairness', 'std')]:.2f})$", axis=1)
    grouped_results = grouped_results.sort_values(by=['Fitness Rule Abvr', 'dataset'])
    grouped_results_list.append(grouped_results)

Model comparisons: 100%|█████████▉| 5994/6000 [00:13<00:00, 430.15it/s]


Significance Testing


Model comparisons: 100%|█████████▉| 5994/6000 [00:13<00:00, 461.06it/s]


Significance Testing


In [63]:
significances[0].sort_values(by=['fitness_rule', 'dataset'], ascending=[False, True])

Standard MLP (baseline)  \
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income                      1.000000   
                        Adult Income                      0.529610   
                        Adult Income                      1.000000   
                        Adult Income                      0.253039   
                        Bank Marketing                    1.000000   
...                                                            ...   
Max(Acc - Eq. Odds)     Compas Recidivism                 1.000000   
                        German Credit                     1.000000   
                        German Credit                     0.208598   
                        German Credit                     0.625273   
                        German Credit                     0.035121   

                                           Pearson regularized MLP  \
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income                      1.000000   
                        Adult Income                      1.000000   
                        Adult Income                      1.000000   
                        Adult Income                      0.704854   
                        Bank Marketing                    1.000000   
...                                                            ...   
Max(Acc - Eq. Odds)     Compas Recidivism                 1.000000   
                        German Credit                     1.000000   
                        German Credit                     1.000000   
                        German Credit                     1.000000   
                        German Credit                     0.134923   

                                           Spearman regularized MLP  \
fitness_rule            dataset                                       
Max(MCC - Stat. Parity) Adult Income                       0.920497   
                        Adult Income                       0.560012   
                        Adult Income                       1.000000   
                        Adult Income                       0.367520   
                        Bank Marketing                     1.000000   
...                                                             ...   
Max(Acc - Eq. Odds)     Compas Recidivism                  1.000000   
                        German Credit                      1.000000   
                        German Credit                      0.264121   
                        German Credit                      1.000000   
                        German Credit                      0.008545   

                                           Auto corr regularized MLP  \
fitness_rule            dataset                                        
Max(MCC - Stat. Parity) Adult Income                        1.000000   
                        Adult Income                        1.000000   
                        Adult Income                        1.000000   
                        Adult Income                        1.000000   
                        Bank Marketing                      0.666231   
...                                                              ...   
Max(Acc - Eq. Odds)     Compas Recidivism                   1.000000   
                        German Credit                       0.991947   
                        German Credit                       1.000000   
                        German Credit                       0.999194   
                        German Credit                       1.000000   

                                                              method  
fitness_rule            dataset                                       
Max(MCC - Stat. Parity) Adult Income         Standard MLP (baseline)  
                        Adult Income         Pearson regularized MLP  
                        Adult Income        Spearman regularized MLP  
     

In [64]:
grouped_results_list[0]

fitness  \
                                                                         mean   
Fitness Rule Abvr       dataset           method                                
Max(Acc - Eq. Odds)     Adult Income      Auto corr regularized MLP  0.765534   
                                          Spearman regularized MLP   0.761024   
                                          Standard MLP (baseline)    0.758290   
                                          Pearson regularized MLP    0.741188   
                        Bank Marketing    Standard MLP (baseline)    0.803182   
...                                                                       ...   
Max(MCC - Stat. Parity) Compas Recidivism Spearman regularized MLP   0.067990   
                        German Credit     Auto corr regularized MLP  0.288807   
                                          Standard MLP (baseline)    0.265661   
                                          Pearson regularized MLP    0.262448   
                                          Spearman regularized MLP   0.180405   

                                                                               \
                                                                          std   
Fitness Rule Abvr       dataset           method                                
Max(Acc - Eq. Odds)     Adult Income      Auto corr regularized MLP  0.022879   
                                          Spearman regularized MLP   0.041571   
                                          Standard MLP (baseline)    0.035161   
                                          Pearson regularized MLP    0.045045   
                        Bank Marketing    Standard MLP (baseline)    0.070894   
...                                                                       ...   
Max(MCC - Stat. Parity) Compas Recidivism Spearman regularized MLP   0.069200   
                        German Credit     Auto corr regularized MLP  0.078940   
                                          Standard MLP (baseline)    0.099801   
                                          Pearson regularized MLP    0.030490   
                                          Spearman regularized MLP   0.088635   

                                                                           \
                                                                    count   
Fitness Rule Abvr       dataset           method                            
Max(Acc - Eq. Odds)     Adult Income      Auto corr regularized MLP     6   
                                          Spearman regularized MLP      6   
                                          Standard MLP (baseline)      15   
                                          Pearson regularized MLP       6   
                        Bank Marketing    Standard MLP (baseline)      30   
...                                                                   ...   
Max(MCC - Stat. Parity) Compas Recidivism Spearman regularized MLP      6   
                        German Credit     Auto corr regularized MLP     6   
                                          Standard MLP (baseline)      30   
                                          Pearson regularized MLP       6   
                                          Spearman regularized MLP      6   

                                                                    Performance  \
                                                                           mean   
Fitness Rule Abvr       dataset           method                                  
Max(Acc - Eq. Odds)     Adult Income      Auto corr regularized MLP    0.851446   
                                          Spearman regularized MLP     0.847835   
                                          Standard MLP (baseline)      0.847864   
                                          Pearson regularized MLP      0.850341   
                        Bank Marketing    Standard MLP (baseline)      0.902154   
...                                                      

In [65]:
significances[1].sort_values(by=['fitness_rule', 'dataset'], ascending=[False, True])

Standard FTL (baseline)  \
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income                      1.000000   
                        Adult Income                      0.539615   
                        Adult Income                      1.000000   
                        Adult Income                      0.331813   
                        Bank Marketing                    1.000000   
...                                                            ...   
Max(Acc - Eq. Odds)     Compas Recidivism                 0.362404   
                        German Credit                     1.000000   
                        German Credit                     1.000000   
                        German Credit                     1.000000   
                        German Credit                     1.000000   

                                           Pearson regularized FTL  \
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income                      1.000000   
                        Adult Income                      1.000000   
                        Adult Income                      1.000000   
                        Adult Income                      0.662353   
                        Bank Marketing                    1.000000   
...                                                            ...   
Max(Acc - Eq. Odds)     Compas Recidivism                 1.000000   
                        German Credit                     0.795095   
                        German Credit                     1.000000   
                        German Credit                     1.000000   
                        German Credit                     1.000000   

                                           Spearman regularized FTL  \
fitness_rule            dataset                                       
Max(MCC - Stat. Parity) Adult Income                       0.364604   
                        Adult Income                       0.211302   
                        Adult Income                       1.000000   
                        Adult Income                       0.095834   
                        Bank Marketing                     1.000000   
...                                                             ...   
Max(Acc - Eq. Odds)     Compas Recidivism                  1.000000   
                        German Credit                      0.000000   
                        German Credit                      0.000000   
                        German Credit                      1.000000   
                        German Credit                      0.450215   

                                           Auto corr regularized FTL  \
fitness_rule            dataset                                        
Max(MCC - Stat. Parity) Adult Income                        1.000000   
                        Adult Income                        1.000000   
                        Adult Income                        1.000000   
                        Adult Income                        1.000000   
                        Bank Marketing                      1.000000   
...                                                              ...   
Max(Acc - Eq. Odds)     Compas Recidivism                   1.000000   
                        German Credit                       0.051738   
                        German Credit                       0.026711   
                        German Credit                       1.000000   
                        German Credit                       1.000000   

                                                              method  
fitness_rule            dataset                                       
Max(MCC - Stat. Parity) Adult Income         Standard FTL (baseline)  
                        Adult Income         Pearson regularized FTL  
                        Adult Income        Spearman regularized FTL  
     

In [66]:
grouped_results_list[1]

fitness  \
                                                                         mean   
Fitness Rule Abvr       dataset           method                                
Max(Acc - Eq. Odds)     Adult Income      Pearson regularized FTL    0.816948   
                                          Standard FTL (baseline)    0.815099   
                                          Auto corr regularized FTL  0.800019   
                                          Spearman regularized FTL   0.796648   
                        Bank Marketing    Pearson regularized FTL    0.842059   
...                                                                       ...   
Max(MCC - Stat. Parity) Compas Recidivism Auto corr regularized FTL  0.219729   
                        German Credit     Pearson regularized FTL    0.298430   
                                          Standard FTL (baseline)    0.256495   
                                          Auto corr regularized FTL  0.236066   
                                          Spearman regularized FTL   0.183352   

                                                                               \
                                                                          std   
Fitness Rule Abvr       dataset           method                                
Max(Acc - Eq. Odds)     Adult Income      Pearson regularized FTL    0.023932   
                                          Standard FTL (baseline)    0.022059   
                                          Auto corr regularized FTL  0.039151   
                                          Spearman regularized FTL   0.036947   
                        Bank Marketing    Pearson regularized FTL    0.037943   
...                                                                       ...   
Max(MCC - Stat. Parity) Compas Recidivism Auto corr regularized FTL  0.045471   
                        German Credit     Pearson regularized FTL    0.061080   
                                          Standard FTL (baseline)    0.089117   
                                          Auto corr regularized FTL  0.110743   
                                          Spearman regularized FTL   0.166189   

                                                                           \
                                                                    count   
Fitness Rule Abvr       dataset           method                            
Max(Acc - Eq. Odds)     Adult Income      Pearson regularized FTL      17   
                                          Standard FTL (baseline)      14   
                                          Auto corr regularized FTL    13   
                                          Spearman regularized FTL     13   
                        Bank Marketing    Pearson regularized FTL       5   
...                                                                   ...   
Max(MCC - Stat. Parity) Compas Recidivism Auto corr regularized FTL    13   
                        German Credit     Pearson regularized FTL       4   
                                          Standard FTL (baseline)       4   
                                          Auto corr regularized FTL    13   
                                          Spearman regularized FTL      4   

                                                                    Performance  \
                                                                           mean   
Fitness Rule Abvr       dataset           method                                  
Max(Acc - Eq. Odds)     Adult Income      Pearson regularized FTL      0.844002   
                                          Standard FTL (baseline)      0.847485   
                                          Auto corr regularized FTL    0.843747   
                                          Spearman regularized FTL     0.843398   
                        Bank Marketing    Pearson regularized FTL      0.882781   
...                                                      